🕺 This ipynb file is draft for aggregation models for free events about IT and programming: webinars, hackathons, broadcasts and courses.

# TODO:
- `start_parser()` загружает базовую информацию со всех стартовых страниц, перечисленных в файле `pages.json`. Результаты сохраняются в словарях `soups` {`url`:html} и `events` формата {`url`:{`key`:`value`}}. Далее для всех перечисленных в `events` ссылок с датой не позднее сегодня, создаётся объект класса `Event`.

- It will be useful make a common method (in common handler class) that make simple text extracting for soup object with 
- Make specific module for yandex features and common handler class
- For hand-written content: update of events may be not safe
- Write parser for pages of yandex events: div class:events-info
    - Handlers for required_event_keys:
        + 'url'
        + 'title'
        + 'date'
        + 'time'
        + 'location': div class="events-info__location"
        + 'online status'
    - Handlers for optional_event_keys
        + 'registration_url': Registration link 
        + 'price'
        + 'tags'
        + 'language'
        + 'speakers'
        + 'themes'
        - 'type'
    - Check if all required values for the event are found

# Strategy
Shortly: first we check the main pages, then event pages

- Load `pages.json` - information about start wepbages: `name`, `url`, `func`
- Load `events.json` - information about current events:
    - `title`,
    - `url` (description),
    - `registration url` (optional, often the same as description url),
    - `date`
    - `time`
    - `location`
    - `online status`
    - `price`
    - `[tags]`
    - `language` (can be checked by lang of theme)
    - `speakers and themes (optional)`
    - `type (optional)`
- Check that events on the main pages are displayed in `events.json`
- If the event is not in the file, we need to add basic information about it on the second stage
- Past events are transfered to `past_events.json`
- On the next stage if any information is missing, we need to supplement it by going to specific url and parse the page
- For registration url we will add utm information

In [1]:
import re
import json
import requests
from datetime import datetime, date
from bs4 import BeautifulSoup

In [4]:
headers = requests.utils.default_headers()
current_date = date.today()

# all start pages 
with open('pages.json') as pages_file:
    pages = json.load(pages_file)

# main events dict-type structure
with open('events.json') as events_file:
    events = json.load(events_file)

# basic characteristic of common fields
with open('fields.json') as fields_file:
    fields = json.load(fields_file)

# keywords for tag recognition
with open('tags.json') as tags_file:
    tags = json.load(tags_file)

# cities where events are registered
with open('cities.txt') as cities_file:
    cities = cities_file.read().split('\n')

# temp dict for storage of soup objects for testing
soups = {}  

date_alias = {'rus':
                 {'weekdays':
                    {'short': ('пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс')},
                  'months':
                    {'gentive': ('января', 'февраля', 'марта',
                                 'апреля', 'мая', 'июня',
                                 'июля', 'августа', 'сентября',
                                 'октября', 'ноября', 'декабря')}
                 }
              }


def get_soup(url):
    '''Returns (with saving) soup objects'''
    if url not in soups:
        page = requests.get(url, headers=headers).text
        soup = BeautifulSoup(page, 'html.parser')
        soups.update({url:soup})
    return soups[url]


def utm_cleaner(url):
    '''Deletes utm tags from url'''
    index = url.find('?utm')
    if index > 0:
        url = url[:index]
    return url
    

def yandex_date_converter(date_string):
    '''Parsing date in format вт, 9 июня'''
    lang = 'rus'
    weekdays = date_alias[lang]['weekdays']['short']
    months = date_alias[lang]['months']['gentive']
    
    # преобразуем дату из строки
    weekday, event_day, month = date_string.split()
    event_day = int(event_day)
    event_month = months.index(month) + 1
    event_weekday = weekdays.index(weekday[:-1])
    event_year = current_date.year
    
    # рассматриваем случай конца года: слишком старые записи не хранят
    # но во второй половине года могут быть записи на следующий год
    if (event_month-current_date.month) < -3:
        event_year += 1

    event_date = date(event_year, event_month, event_day)
    if event_date.weekday() == event_weekday:
        return event_date.isoformat()

def add_home_url(event_url, start_url):
    ''' Sometimes urls has relative paths'''
    if 'http' not in event_url:
        event_url = start_url[:-1] + event_url
    return(event_url)

#! Необходимо заменить на общий класс обработки начальных страниц
# переопределяемых для конкретных стартовых страниц
# использовать selenium
def yandex(name='Яндекс', start_url='https://events.yandex.ru/'):
    print(f'Parsing page {name} at {start_url}')
    soup = get_soup(start_url)
    event_cards = soup.body.find_all('div', {'class':'event-card'})
    for event_card in event_cards:
        date_string = event_card.find('div', {'class':'event-card__date'}).text
        event_date = yandex_date_converter(date_string)
        event_url = event_card.a['href']
        add_home_url(event_url, start_url)
        print(event_url)
        events[event_url] = {'start':name,
                             'event_url':event_url,
                             'title':event_card.a['title'],
                             'date':event_date}

yandex()

Parsing page Яндекс at https://events.yandex.ru/
https://events.yandex.ru/events/yasubbotnik/4-jul-2020
https://cloud.yandex.ru/events/145
https://cloud.yandex.ru/events/143
https://events.yandex.ru/events/zen-meetup/26-06-2020
https://events.yandex.ru/events/bugs-busters-25-06-2020
https://cloud.yandex.ru/events/138
https://cloud.yandex.ru/events/140
https://events.yandex.ru/events/hardware/19-june-2020


In [57]:
soups['https://events.yandex.ru/'].find_all('div')

[<div id="mount"><div class="header"><div class="header__cover"></div><div class="header__wrapper"><div class="header__logo"><a href="https://yandex.ru"><img alt="Яндекс" src="https://yastatic.net/q/logoaas/v1/Яндекс.svg?color=fff"/></a></div><div class="header__hamburger"></div><div class="header__user"><div class="account"><a class="account__auth" href="https://passport.yandex.ru/auth?retpath=https://events.yandex.ru/">Войти</a></div></div><div class="header__menu"><a class="header__item header__item_active" href="https://events.yandex.ru">События</a><a class="header__item" href="https://research.yandex.com" rel="noopener noreferrer external" target="_blank">Наука</a><a class="header__item" href="https://academy.yandex.ru" rel="noopener noreferrer external" target="_blank">Обучение</a></div></div></div><div class="page-container"><div class="page page_type_home"><h1>Мероприятия Яндекса</h1><div class="filters"><div class="filters__container"><div class="filters-item filters-item_them

In [10]:
with open('events.json', 'w', encoding='utf8') as events_file:
    json.dump(events, events_file, ensure_ascii=False)

# Let's write Event class and handlers for all fields

In [35]:
all_fields = set(fields.keys())
required   = {key for key in fields if fields[key]['required']}
optional   = {key for key in fields if not fields[key]['required']}
analysis   = {key for key in fields if fields[key]['content_type'] == 'analysis'}


def time_handler(time, start):
    if start == 'Яндекс':
        pattern = re.compile(r'\d{2}:\d{2}')
        time = pattern.search(time)
        time = time.group()
        return time


def online_status_handler(online_status, start):
    if status in ('Online', 'Будет трансляция',
                  'Прямая трансляция', 'Прямой эфир'):
        return 'Online'
    else:
        return 'Offline'


def get_content(url, field):
    start        = events[url]['start']
    selectors    = pages[start][field]['selectors']
    default      = pages[start][field].get('default', '')
    elements     = fields[field]['elements']
    content_type = fields[field]['content_type']
    soup         = get_soup(url)
    overlaps     = []
    
    for key in selectors:
        tag, attr = selectors[key]
        overlaps += soup.find_all(tag, {attr:key})
    
    if content_type == 'text':
        content = [overlap.text.strip() for overlap in overlaps]
    elif content_type == 'url':
        content = [overlap['href'] for overlap in overlaps]

    if elements != 'all':
        content = content[:elements]

    if (len(content) == 0) & (elements == 1):
        content = '' or default
    elif (len(content) == 1) & (elements == 1):
        content = content[0]
    
    if (len(content) == 0) & (elements == 'all'):
        content = [default]
    
    # If it's still need processing, let's use handlers
    #content = eval(f'{field}_handler(content, start)')
    return content


class Event:
    def __init__(self, url):
        self.data = events[url]
        self.data = self.update(url)
        
    def update(self, url):
        today = current_date.isoformat()
        available = set(self.data.keys())
        if self.data['date'] >= today:
            if any(required - available):
                fields = all_fields - available - analysis
                for field in fields:
                    print(url)
                    print(field)
                    print(get_content(url, field))
                    self.data[field] = get_content(url, field)
                #! then add analysis fields processing 
                #! for example - eval with functions
                tags()

    
    def tags(self):
        self.data[tags] = {}
        themes = self.data[themes].copy().append(title).append(url)
        text = ''.join(themes)
        for key in tags:
            for tag in tags[key]:
                if tag.lower() in text.lower():
                    self.data[tags].add(tag)
        return self.data[tags]

    
    def display(self):
        print(self.data)

In [44]:
event = Event('https://events.yandex.ru/events/yasubbotnik/4-jul-2020')

In [49]:
events

{'https://events.yandex.ru/events/yasubbotnik/4-jul-2020': {'start': 'Яндекс',
  'event_url': 'https://events.yandex.ru/events/yasubbotnik/4-jul-2020',
  'title': 'Я.Субботник по разработке интерфейсов. Онлайн, темы выбираете вы!',
  'date': '2020-07-04',
  'location': '',
  'organizers': ['Яндекс'],
  'online_status': 'Будет трансляция',
  'themes': [''],
  'time': '4 июля, 12:00 - 16:00 по Москве'},
 'https://cloud.yandex.ru/events/145': {'start': 'Яндекс',
  'event_url': 'https://cloud.yandex.ru/events/145',
  'title': 'about:cloud – бессерверные технологии и IoT',
  'date': '2020-07-02'},
 'https://cloud.yandex.ru/events/143': {'start': 'Яндекс',
  'event_url': 'https://cloud.yandex.ru/events/143',
  'title': 'Вебинар. Как работает сеть в Облаке',
  'date': '2020-06-30'},
 'https://events.yandex.ru/events/zen-meetup/26-06-2020': {'start': 'Яндекс',
  'event_url': 'https://events.yandex.ru/events/zen-meetup/26-06-2020',
  'title': 'Дзен-митап: видеорекомендации. Онлайн',
  'date': '

# Let's process HTML and deploy the current version of the project 💃

In [27]:
soup = BeautifulSoup()
section = soup.new_tag('section')
section.string = "new test"
soup.append(section)
str(soup)

'<section>new test</section>'

In [28]:
content = str(soup) # for test

from pathlib import Path
path_to_html_template = Path("../matyushkin.github.io/11ty/_includes/events.njk")

with open(path_to_html_template, 'wt', encoding='utf-8') as html_template:
    html_template.write(content)

I use [11ty](https://www.11ty.dev/) processing for manipulations with templates, headers and footers of html files on my [web site](https://matyushkin.github.io/). So the simplest way to include and deploy new files at GitHub pages is a Jupyter cell with simple bash code. I use `;` operator because Jupyter returns to current pwd on a new line.

In [29]:
!cd ../matyushkin.github.io/11ty/; npx eleventy --passthroughall --output=..

# Let's see quickly what we have now
import webbrowser
url = "../matyushkin.github.io/events/index.html"
webbrowser.open_new_tab(url)

Writing ../404/index.html from ./404.html.
Writing ../README/index.html from ./README.md.
Writing ../index.html from ./index.html.
Writing ../cv/index.html from ./cv/index.html.
Writing ../donate/index.html from ./donate/index.html.
Writing ../events/index.html from ./events/index.html.
Writing ../links/index.html from ./links/index.html.
Writing ../spb/index.html from ./spb/index.html.
Writing ../posts/index.html from ./posts/index.html.
Writing ../posts/source/template/index.html from ./posts/source/template.html.
Copied 20 files / Wrote 10 files in 0.17 seconds (17.0ms each, v0.11.0)


True

If all is OK, it's only one line for deployment at GitHub pages:

In [ ]:
! cd ../matyushkin.github.io/; git add . ; git commit -m "Events project files are updated"; git push origin master